In [110]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.misc import imsave, imresize
from scipy.optimize import fmin_l_bfgs_b
from keras.applications import vgg16
from keras import backend as K
from tensorflow.contrib.keras.api.keras.preprocessing.image import load_img, img_to_array

In [111]:
# load image
def load_image(filename, dims=(224, 224, 3)):
    img = load_img(filename)
    img = img_to_array(img)
    img = imresize(img, dims)
    img = img.astype('float64')
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

def restore_image(img, img_h=224, img_w=224):
    img = img.reshape((img_h, img_w, 3))
    # Remove zero-center by mean pixel
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [112]:
# save image
def save_image(image, filename):
    image = np.clip(image, 0.0, 255.0)
    image = image.astype(np.uint8)
    
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [114]:
# loss functions
def mean_squared_error(a, b):
    return K.mean(K.square(a - b))

def gram_matrix(tensor, num_channels=3):
    gram = K.dot(K.transpose(tensor), tensor)
    return gram
    
def content_loss(model, content_layer_ids):
    total_loss = 0
    
    for layer_id in content_layer_ids:
        layer_features = model.get_layer(index=layer_id).output
        
        content_features = layer_features[0, :, :, :]
        combination_features = layer_features[2, :, :, :]
        
        loss = mean_squared_error(content_features, combination_features)
        
        total_loss += loss
    
    return total_loss

def style_loss(model, style_layer_ids):
    total_loss = 0

    for layer_id in style_layer_ids:
        layer_features = model.get_layer(index=layer_id).output
        
        style_features = gram_matrix(layer_features[1, :, :, :])
        combination_features = gram_matrix(layer_features[2, :, :, :])
        
        loss = mean_squared_error(style_features, combination_features)
        
        total_loss += loss
    
    return total_loss

def denoise_loss(x):
    loss = K.sum(K.abs(x[:,1:,:,:] - x[:,:-1,:,:])) + K.sum(K.abs(x[:,:,1:,:] - x[:,:,:-1,:]))
    return loss

In [118]:
# style transfer
def style_transfer(content_image, style_image, content_layer_ids, style_layer_ids, 
                   weight_content=1.5, weight_style=10.0, weight_denoise=0.3, num_iterations=120,
                  step_size=10.0, dims=(224,224,3)):
    
    content_image = K.variable(content_image)
    style_image = K.variable(style_image)
    combination_image = K.placeholder(shape=(1, dims[0], dims[1], dims[2]))
    
    
    input_tensor = K.concatenate([content_image, style_image, combination_image], axis=0)
    
    model = vgg16.VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)
    
        
    content_loss_val = content_loss(model, content_layer_ids) 
    style_loss_val = style_loss(model, style_layer_ids)    
    
    loss_combined = weight_content * content_loss_val + weight_style * style_loss_val
    
    grads = K.gradients(loss_combined, combination_image)
    
    outputs = [loss_combined]
    
    if isinstance(grads, (list, tuple)):
        outputs += grads
    else:
        outputs.append(grads)
        
    f_output = K.function([combination_image], outputs)

    def get_loss(x, img_w=dims[0], img_h=dims[1]):
        x = x.reshape((1, img_h, img_w, 3))
        outs = f_output([x])
        return outs[0]
    
    def get_grads(x, img_w=dims[0], img_h=dims[1]):
        x = x.reshape((1, img_h, img_w, 3))
        outs = f_output([x])
        if len(outs[1:]) == 1:
            grad_values = outs[1].flatten().astype('float64')
        else:
            grad_values = np.array(outs[1:]).flatten().astype('float64')
        return grad_values
    
    return get_loss, get_grads


In [119]:
content_filename = 'resources/willy_wonka_old.jpg'
content_image = load_image(content_filename)
style_filename = 'resources/style1.jpg'
style_image = load_image(style_filename)
content_layer_ids = [4]
style_layer_ids = list(range(13))

/Users/pedrocardoso/.local/share/virtualenvs/mlnotes--cLpeW_k/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


In [ ]:
get_loss, get_grads = style_transfer(content_image=content_image, style_image=style_image, 
               content_layer_ids=content_layer_ids, style_layer_ids=style_layer_ids)



x, min_val, info = fmin_l_bfgs_b(get_loss, content_image.flatten(), fprime=get_grads, maxiter=1)
print('loss: {}'.format(min_val))
# Save img
img = restore_image(x)
imsave('/img{}.jpg'.format(1), img)